# 1. Tổng quan về VGI và VTL

### 1. **FPT Corporation (FPT)**
   - **Ngành nghề**: Công nghệ thông tin và Viễn thông.
   - **Mô tả**: FPT là tập đoàn công nghệ hàng đầu tại Việt Nam, hoạt động trong nhiều lĩnh vực như phần mềm, dịch vụ IT, viễn thông, và giáo dục. FPT có mạng lưới khách hàng rộng lớn trong nước và quốc tế.

### 2. **CMC Corporation (CMG)**
   - **Ngành nghề**: Công nghệ thông tin và Viễn thông.
   - **Mô tả**: CMC là một trong những tập đoàn công nghệ lớn tại Việt Nam, chuyên cung cấp các giải pháp về phần mềm, hạ tầng IT, và an ninh mạng. CMC cạnh tranh trực tiếp với FPT trong một số lĩnh vực.



In [2]:
!pip install pandas numpy scipy matplotlib


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# Đọc và xử lý dữ liệu
def process_stock_data(df):
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df

# Tính toán các metrics
def calculate_metrics(df):
    df['daily_returns'] = df['close'].pct_change()
    df['volatility'] = df['daily_returns'].rolling(window=20).std() * np.sqrt(252)
    df['MA20'] = df['close'].rolling(window=20).mean()
    df['MA50'] = df['close'].rolling(window=50).mean()
    return df

# Load dữ liệu
vgi_df = process_stock_data(pd.read_csv('../data/raw/VGI_stock_data.csv'))
vtl_df = process_stock_data(pd.read_csv('../data/raw/VTL_stock_data.csv'))

# Tính toán metrics cho cả hai cổ phiếu
vgi_df = calculate_metrics(vgi_df)
vtl_df = calculate_metrics(vtl_df)

# Tính correlation
correlation = vgi_df['close'].corr(vtl_df['close'])

# Đồng bộ hóa dữ liệu giữa VGI và VTL
common_dates = vgi_df.index.intersection(vtl_df.index)  # Lấy các ngày chung
vgi_returns = vgi_df.loc[common_dates, 'daily_returns'].dropna()
vtl_returns = vtl_df.loc[common_dates, 'daily_returns'].dropna()
# Tính beta của VTL so với VGI
beta = np.cov(vtl_returns, vgi_returns)[0, 1] / np.var(vgi_returns)

# Tính toán thống kê cơ bản
def calculate_statistics(df, name):
    stats_dict = {
        'Mean Price': df['close'].mean(),
        'Std Dev': df['close'].std(),
        'Min Price': df['close'].min(),
        'Max Price': df['close'].max(),
        'Avg Volume': df['volume'].mean(),
        'Avg Daily Return': df['daily_returns'].mean() * 100,
        'Volatility': df['daily_returns'].std() * np.sqrt(252) * 100
    }
    return pd.Series(stats_dict, name=name)

vgi_stats = calculate_statistics(vgi_df, 'VGI')
vtl_stats = calculate_statistics(vtl_df, 'VTL')

# In kết quả phân tích
print("\nCorrelation between VGI and VTL:", correlation)
print("\nBeta of VTL relative to VGI:", beta)
print("\nStatistical Analysis:")
print(pd.concat([vgi_stats, vtl_stats], axis=1))

# Tính toán spread
common_dates = vgi_df.index.intersection(vtl_df.index)
price_spread = vgi_df.loc[common_dates, 'close'] - vtl_df.loc[common_dates, 'close']
spread_zscore = stats.zscore(price_spread)

# Tính % thời gian spread vượt quá 2 std
spread_threshold = np.abs(spread_zscore) > 2
spread_anomaly_pct = (spread_threshold.sum() / len(spread_zscore)) * 100

print(f"\nSpread Analysis:")
print(f"Mean Spread: {price_spread.mean():.2f}")
print(f"Spread Std Dev: {price_spread.std():.2f}")
print(f"% time spread > 2 std: {spread_anomaly_pct:.2f}%")



Correlation between VGI and VTL: -0.921582218742406

Beta of VTL relative to VGI: -0.058818542376283166

Statistical Analysis:
                            VGI           VTL
Mean Price         41050.888889  11904.859335
Std Dev            25712.657731   3476.278871
Min Price          19900.000000   4500.000000
Max Price         111000.000000  15300.000000
Avg Volume        960421.504444   1198.567775
Avg Daily Return       0.291010     -0.072280
Volatility            45.546809     85.345609

Spread Analysis:
Mean Spread: 26383.38
Spread Std Dev: 29123.99
% time spread > 2 std: 9.72%


## 1.2 Nhận xét

### **Phân tích kết quả cho cặp VGI và VTL**

---

#### **1. Tương quan (Correlation)**

- **Hệ số tương quan**: **-0.922**
  - Tương quan âm rất mạnh, nghĩa là khi giá cổ phiếu **VGI tăng**, giá **VTL giảm** và ngược lại.
  - Đây là cơ hội tốt cho **chiến lược giao dịch ngược chiều (reversal trading)**, nơi ta có thể mua một cổ phiếu và bán cổ phiếu kia.

---

#### **2. Beta của VTL so với VGI**

- **Beta**: **-0.059**
  - Beta âm nhỏ cho thấy:
    - Mỗi khi **VGI** thay đổi 1%, **VTL** thay đổi **0.059% theo hướng ngược lại**.
    - Độ nhạy giữa hai cổ phiếu thấp, điều này có nghĩa là giá VTL ít bị ảnh hưởng trực tiếp bởi biến động của VGI.

---

#### **3. Thống kê cơ bản**

| **Chỉ số**               | **VGI**          | **VTL**          | **Nhận xét**                                                                 |
|--------------------------|------------------|------------------|------------------------------------------------------------------------------|
| **Mean Price**           | 41,051 đồng      | 11,905 đồng      | VGI có giá trung bình cao hơn gần **3.5 lần** so với VTL.                    |
| **Std Dev (Độ lệch chuẩn)** | 25,713 đồng   | 3,476 đồng       | VGI dao động mạnh hơn, trong khi VTL ổn định hơn về giá trị tuyệt đối.       |
| **Volatility (Biến động)** | 45.55%          | 85.35%           | VTL biến động mạnh hơn, là cổ phiếu rủi ro cao nhưng có tiềm năng lợi nhuận. |
| **Avg Daily Return (%)** | 0.291%           | -0.072%          | VGI có tỷ suất sinh lời trung bình hàng ngày tốt hơn đáng kể.                |

---

#### **4. Phân tích Spread**

- **Mean Spread**: **26,383 đồng**
  - Trung bình giá **VGI** cao hơn giá **VTL** khoảng 26,383 đồng.

- **Spread Std Dev**: **29,124 đồng**
  - Độ lệch chuẩn của spread khá lớn, cho thấy khoảng cách giá giữa VGI và VTL biến động mạnh.

- **% Time Spread > 2 Std Dev**: **9.72%**
  - Gần 10% thời gian, spread vượt quá 2 độ lệch chuẩn, tạo ra **cơ hội giao dịch** khi khoảng cách giá bất thường.

---

### **Đề xuất chiến lược giao dịch**

1. **Chiến lược Reversal Trading**:
   - Khi giá VGI tăng mạnh (spread tăng), cân nhắc **bán VGI và mua VTL**.
   - Khi giá VGI giảm mạnh (spread giảm), cân nhắc **mua VGI và bán VTL**.

2. **Ngưỡng giao dịch**:
   - **Entry Point**: Khi spread vượt quá **2 Std Dev** (≈ 85,631 đồng).
   - **Exit Point**: Khi spread quay về mức trung bình (≈ 26,383 đồng).
   - **Stop Loss**: Khi spread vượt quá **3 Std Dev** (≈ 114,755 đồng).

3. **Lưu ý quản trị rủi ro**:
   - **VTL** có biến động mạnh (Volatility 85%), cần quản lý vị thế để tránh rủi ro lớn.
   - Sử dụng tỷ lệ vốn nhỏ hơn cho VTL so với VGI.

---

### **Kết luận**
- **VGI và VTL** là một cặp lý tưởng cho **chiến lược giao dịch ngược chiều** nhờ tương quan âm mạnh (-0.922) và spread biến động lớn.
- **Ưu tiên giao dịch trong ngắn hạn**, vì cả hai cổ phiếu đều có mức độ biến động cao.
- Cần theo dõi spread chặt chẽ và sử dụng công cụ quản lý rủi ro như **Stop Loss** để đảm bảo an toàn. 

## 1.3 Cùng phân tích để hiểu hơn:


### **1. Entry Point, Exit Point, Stop Loss là gì?**

#### **Entry Point (Điểm vào lệnh)**:
- **Là thời điểm bắt đầu mua hoặc bán cổ phiếu.**
- Trong chiến lược này, khi **spread (chênh lệch giá)** giữa hai cổ phiếu vượt **2 độ lệch chuẩn (std dev)**, đó là **cơ hội để vào lệnh**.
- Ví dụ: Nếu giá VGI cao hơn VTL một cách bất thường (spread lớn), bạn sẽ **bán VGI và mua VTL**.

#### **Exit Point (Điểm thoát lệnh)**:
- **Là thời điểm kết thúc giao dịch, chốt lời hoặc cắt lỗ.**
- Trong chiến lược này, khi **spread trở về mức trung bình (Mean Spread)**, bạn sẽ thoát lệnh.
- Mục tiêu: Đợi giá quay về trạng thái bình thường để chốt lời.

#### **Stop Loss (Giới hạn lỗ)**:
- **Là mức lỗ tối đa bạn chấp nhận được.**
- Trong chiến lược này, nếu **spread vượt quá 3 độ lệch chuẩn**, bạn nên dừng giao dịch để tránh lỗ lớn.
- Mục tiêu: Bảo vệ vốn khi thị trường biến động ngoài dự đoán.

---

### **2. Phân tích chiến lược đầu tư cho học sinh cấp 2**

Hãy tưởng tượng bạn có hai người bạn chơi cầu trượt ở công viên:
- **VGI**: Bạn A, thích chạy nhanh và trượt xa.
- **VTL**: Bạn B, lúc trượt nhanh, lúc trượt chậm và ngược hướng với bạn A.

#### **Mối quan hệ giữa hai bạn này (VGI và VTL):**
- Khi **bạn A (VGI) lên dốc**, thường **bạn B (VTL) xuống dốc**.
- Khi **bạn B (VTL) chạy xa**, bạn A thường quay lại gần hơn.

#### **Chiến lược đầu tư giống như chơi trò đoán hướng di chuyển:**
1. **Quan sát khoảng cách (spread)** giữa hai bạn:
   - Nếu khoảng cách giữa bạn A và bạn B lớn bất thường (Entry Point): **Bạn đặt cược rằng khoảng cách sẽ thu hẹp**.
   - Ví dụ: Mua bạn B (VTL) và bán bạn A (VGI).

2. **Khi hai bạn quay lại gần nhau (Exit Point)**:
   - Bạn thắng cược và kết thúc trò chơi, vì khoảng cách đã trở lại bình thường.

3. **Nếu khoảng cách giữa hai bạn lớn hơn cả mức an toàn (Stop Loss)**:
   - Bạn thua cược và phải dừng chơi để không mất thêm tiền.

---

### **Ví dụ dễ hiểu về chiến lược này**

#### **1. Khi Spread vượt 2 Std Dev (Entry Point):**
- Spread (chênh lệch giá) giữa VGI và VTL quá lớn.
- **Hành động**:
  - **Bán VGI** (vì giá đang quá cao).
  - **Mua VTL** (vì giá đang quá thấp).

#### **2. Khi Spread về Mean Spread (Exit Point):**
- Spread trở lại bình thường.
- **Hành động**:
  - **Mua lại VGI** (để đóng lệnh bán).
  - **Bán lại VTL** (để đóng lệnh mua).
- Bạn chốt lời từ sự thay đổi chênh lệch giá.

#### **3. Khi Spread vượt 3 Std Dev (Stop Loss):**
- Spread lớn hơn mức dự đoán.
- **Hành động**:
  - Dừng giao dịch, không tiếp tục để tránh mất thêm tiền.

---

### **Kết luận dễ hiểu**
- **Chiến lược này giống như chơi trò đoán khoảng cách**:
  - Khi hai người bạn (VGI và VTL) trượt cầu trượt quá xa, bạn dự đoán họ sẽ quay lại gần nhau.
  - Bạn "đặt cược" vào sự thay đổi khoảng cách này để kiếm lời.

- **Hãy luôn đặt giới hạn lỗ (Stop Loss)** để không thua quá nhiều khi đoán sai.

Bằng cách này, bạn sẽ hiểu cách sử dụng **spread, entry point, exit point, và stop loss** để đầu tư! 😊

# 2. **So sánh sự khác nhau giữa Pair Trading và Trading Reversal**


### **So sánh sự khác nhau giữa Pair Trading và Trading Reversal**

| **Tiêu chí**                  | **Pair Trading**                                                                                          | **Trading Reversal**                                                                                          |
|--------------------------------|----------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------|
| **Định nghĩa**                 | Giao dịch dựa trên **mối tương quan cùng chiều** giữa hai cổ phiếu.                                        | Giao dịch dựa trên **mối tương quan ngược chiều** giữa hai cổ phiếu.                                           |
| **Nguyên tắc hoạt động**       | - Khi giá của hai cổ phiếu có chênh lệch bất thường, bạn kỳ vọng khoảng cách này sẽ trở lại mức trung bình. | - Khi một cổ phiếu tăng mạnh, cổ phiếu kia có xu hướng giảm và ngược lại.                                      |
| **Tương quan**                 | **Tương quan dương cao** (gần 1): Hai cổ phiếu thường di chuyển cùng chiều.                              | **Tương quan âm mạnh** (gần -1): Hai cổ phiếu di chuyển ngược chiều.                                          |
| **Mục tiêu giao dịch**         | Tận dụng sự **chênh lệch giá tạm thời** giữa hai cổ phiếu để kiếm lời khi giá quay về trạng thái bình thường. | Tận dụng **xu hướng ngược chiều** để kiếm lời khi giá cổ phiếu thay đổi theo quy luật ngược chiều.            |
| **Điểm vào lệnh (Entry Point)** | - Khi chênh lệch giá (spread) vượt quá mức bất thường, ví dụ: ±2 độ lệch chuẩn.                          | - Khi một cổ phiếu tăng hoặc giảm mạnh (ví dụ: vượt mức ±2 độ lệch chuẩn so với giá trung bình).               |
| **Điểm thoát lệnh (Exit Point)** | - Khi spread quay về mức trung bình.                                                                    | - Khi cổ phiếu quay lại trạng thái cân bằng hoặc đạt kỳ vọng ngược chiều.                                     |
| **Rủi ro**                     | - Phụ thuộc vào **sự duy trì tương quan** giữa hai cổ phiếu.                                             | - Rủi ro cao hơn do biến động mạnh hơn, đặc biệt nếu tương quan yếu đi.                                        |
| **Ví dụ cổ phiếu**             | **FPT - CMG**: Hai cổ phiếu công nghệ cùng ngành, thường tăng/giảm cùng chiều.                          | **VGI - VTL**: Hai cổ phiếu có tương quan âm, thường di chuyển ngược hướng nhau.                              |
| **Phù hợp với nhà đầu tư**      | - **Người ưa thích an toàn**, vì mức biến động thường thấp và chiến lược dựa trên mối tương quan ổn định.  | - **Người chấp nhận rủi ro**, vì chiến lược này tận dụng biến động mạnh giữa hai cổ phiếu.                     |
| **Yêu cầu quản lý vốn**         | - Quản lý rủi ro chặt chẽ hơn khi spread biến động lớn bất thường.                                       | - Quản lý vốn cẩn thận hơn, do khả năng thua lỗ cao hơn nếu đoán sai hướng đảo chiều.                         |
| **Lợi thế**                    | - An toàn hơn do thường giao dịch dựa trên mối tương quan mạnh.                                          | - Có thể kiếm lời nhanh chóng nhờ sự khác biệt rõ ràng giữa hai cổ phiếu trong xu hướng ngược chiều.          |

---

### **Tóm lại:**
- **Pair Trading**:
  - Phù hợp với các cổ phiếu có tương quan dương cao.
  - Tập trung vào việc kiếm lời từ **khoảng cách giá tạm thời** giữa hai cổ phiếu.

- **Trading Reversal**:
  - Phù hợp với các cổ phiếu có tương quan âm mạnh.
  - Tận dụng sự **ngược chiều rõ ràng** để giao dịch. 
